# Simulation einer Batterie

Ihr müsst eine "Zelle" ausführen. Hierzu einfach den kleinen play button drücken oder Umschalt+Enter wen ihr in einen Block klickt unten könnt Ihr dann Werte einsetzen

Hierfür müsst Ihr PyBAMM installieren
das geht via 
```
pip install pybamm
```


In [ ]:
#ein paar importe damit wir loslegen können
import pybamm
import matplotlib.pyplot as plt

Im folgenden werden wir ein ziemliches Standartexperiment in der Baterieforschung machen: das sogenannte CCCV experiment. Zuerst legt man einen konstanten Strom (constant current - CC) und dann eine konstante spannung (constant voltage - CV). Im Grunde handelt es sich hierbei um zuerst ein Chronopotentiometrisches und dann um ein Chronoamperometrisches Messverfahren.\
Idr gibt man den Ladestrom in einer C-Rate an d.h. ein 1C Strom Läd die Batterie in Einer Stunde d.h. wenn es eine Batterie wäre mit 3.4Ah dann wäre eine 1C Rate eben 3.4A. Wenn man 5C macht dementsprechen der Strom der die Batterie theoretisch in 1/5 Stunde - also 12 min - laden würde usw. Läd man Langsamer Schreibt man idr C/6 also eine volle Ladung in 6-Stunden. Man kann aber auch 1/6C schreiben.\
In CV part legt man die maximale Spannung an die noch sicher ist - hier nun 4.1V. Das mach man so lange bis ein gewisser minimaler Strom Erreicht ist. Ihr könnt euch das so vorstellen das die Batterie im CC teil auf ca. 95% oder so läd und man dann im CV part auf 100% läd. Das schauen wir uns noch genauer an!\
Damit die Zelle nicht zu warm wird sollte man sie ein wenig ruhen lassen.\
Im folgenden machen wir nun einen langsamen Lade- und Entladezyklus!

In [ ]:
pybamm.set_logging_level("INFO")
experiment = pybamm.Experiment(
    [
        (
            "Charge at C/20 until 4.1 V",
            "Hold at 4.1 V until C/50",
            "Rest for 1 hour",
            "Discharge at C/20 until 3.3 V",

        )
    ]
)
model = pybamm.lithium_ion.DFN()

sim = pybamm.Simulation(
    model, experiment=experiment, solver=pybamm.CasadiSolver("fast with events")
)
sim.solve(initial_soc=0.0)

Wir plotten nun das Ergebnis und schauen uns den Spannungsverlauf genauer an!

In [ ]:
t = sol["Time [h]"].entries
V = sol["Terminal voltage [V]"].entries
I = sol["Current [A]"].entries

plt.plot(t,V,label="C/20")
plt.legend()
plt.xlabel("Time [h]")
plt.xlabel("Potential [V]")
plt.show()

In der Vorlesung werden wir das einwenig genauer diskutieren aber schaut euch qualitativ den Verlauf an!
Ein Paar Fragen zum überlegen:\
1) Bis zu welchen state of charge (SOC) laden wir in etwa im CC teil?
2) Warum macht die Spannung solche "Wellen"?

# Vergleich zu LFP
Lithium Eisen Phosphat ist eine sehr günstige und umweltfreundliche Alternative zu NMC. Wir simulieren mal eine solche zelle!

In [ ]:

#let's look at LFP
parameter_values = pybamm.ParameterValues("Prada2013")

experiment = pybamm.Experiment(
    [
        (
            "Charge at C/20 until 3.9 V",
            "Hold at 3.9 V until C/50",
            "Rest for 1 hour",
            "Discharge at C/20 until 2.1 V",

        )
    ]
)

sim = pybamm.Simulation(
    model, experiment=experiment, solver=pybamm.CasadiSolver("safe"),parameter_values=parameter_values
)
sol = sim.solve(initial_soc=0.00)

t = sol["Time [h]"].entries
V = sol["Terminal voltage [V]"].entries
I = sol["Current [A]"].entries

plt.plot(t,V,label="C/20")
plt.legend()
plt.xlabel("Time [h]")
plt.xlabel("Potential [V]")
plt.show()

Was fällt euch auf? Habt ihr eine Ahnung warum es ein Plateau gibt?

# Ein bisschen schnell laden!

Wir schauen uns mal an wie das Ladeverhalten bei unterschiedlichen C-raten aussieht

In [ ]:

raten = [1/24, 1/10, 1/5, 1]
for rate in raten:
    print(rate)
    experiment = pybamm.Experiment(
        [
            (
                "Charge at {}C until 3.9 V".format(rate),
                "Hold at 3.9 V until C/50",
                "Rest for 1 hour",
                "Discharge at {}C until 2.1 V".format(rate),
            )
        ]
    )

    sim = pybamm.Simulation(
        model, experiment=experiment, solver=pybamm.CasadiSolver("safe")
    )
    sol = sim.solve(initial_soc=0.001)

    t = sol["Time [h]"].entries
    V = sol["Terminal voltage [V]"].entries
    I = sol["Current [A]"].entries
    capa = -sol['Discharge capacity [A.h]'].entries
    plt.plot(capa,V,label="{}C".format(rate))
    plt.legend()
    plt.xlabel("Capacity [A.h]")
    plt.ylabel("Potential [V]")
plt.show()

Schaut euch das ergebnis an und überlegt mal welche form des ladens besonders energie (in)effizient ist!

Nur um ein paar Zahlen zu nennen: Mein Auto hatte 77kWh und wurde von mir auf dem Weg von/nach Karlsruhe meistens einmal kurz mit 250kW geladen -> Was ist das in etwa für eine C-rate und wie effizient ist das (Akkukühlung ausgenommen!)?